In [1]:
import warnings
import logging

root_logger = logging.getLogger()
root_logger.setLevel(logging.ERROR)
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import random
from faker import Faker

# 初始化Faker库用于生成随机文本
fake = Faker('zh_CN')

# 定义用户A和用户B的对话模板
user_a_responses = [
    "别担心，一切都会好起来的。",
    "你已经做得很棒了，继续加油！",
    "无论遇到什么困难，我都会在你身边。",
    "相信自己，你一定可以克服这个挑战。",
    "每一次挫折都是成长的机会，别灰心。",
    "慢慢来，不要给自己太大压力。",
    "你已经比昨天进步了很多，继续前进吧。",
    "我会一直支持你，无论发生什么。",
    "困难只是暂时的，未来会更美好。",
    "你比你想象的要坚强得多。"
]

user_b_messages = [
    "我感觉很迷茫，不知道该怎么办。",
    "最近工作压力很大，有点撑不住了。",
    "我好像总是做不好，很沮丧。",
    "我不知道该怎么面对这个挑战。",
    "我感觉自己很失败，很无助。",
    "我不知道该怎么继续下去了。",
    "我觉得自己好像什么都做不好。",
    "我感觉很累，有点想放弃了。",
    "我不知道该怎么解决这个问题。",
    "我觉得自己好像被困住了。"
]

# 生成500条聊天记录
chat_records = []
for _ in range(500):
    user_b_message = random.choice(user_b_messages)
    user_a_response = random.choice(user_a_responses)
    chat_records.append({
        'timestamp': fake.date_time_between(start_date='-1y', end_date='now').strftime('%Y-%m-%d %H:%M:%S'),
        'sender': '用户B',
        'message': user_b_message,
        'message_type': 'text'
    })
    chat_records.append({
        'timestamp': fake.date_time_between(start_date='-1y', end_date='now').strftime('%Y-%m-%d %H:%M:%S'),
        'sender': '用户A',
        'message': user_a_response,
        'message_type': 'text'
    })

# 将聊天记录转换为DataFrame
df = pd.DataFrame(chat_records)

# 保存为CSV文件
df.to_csv('/data/chats/mn0vvd/workspace/chat_records.csv', index=False, encoding='utf-8-sig')

In [3]:
import pandas as pd
import json

# 读取CSV文件
df = pd.read_csv('/data/chats/mn0vvd/workspace/chat_records.csv')

# 过滤出文本类型的消息
df = df[df['message_type'] == 'text']

# 初始化对话配对列表
paired_data = []

# 遍历数据，配对用户A和用户B的对话
prev_user, prev_message = None, None
for _, row in df.iterrows():
    if row['sender'] == '用户B' and prev_user == '用户A':
        paired_data.append({'input': prev_message, 'response': row['message']})
    prev_user, prev_message = row['sender'], row['message']

# 将配对后的对话保存为JSON文件，用于训练
with open('/data/chats/mn0vvd/workspace/training_data.json', 'w', encoding='utf-8') as f:
    json.dump(paired_data, f, ensure_ascii=False, indent=4)

# 调用腾讯云DeepSeek API进行训练
import requests

url = "https://tencent_api_gateway/deepseek/train"
headers = {
    "X-API-Key": "your_api_key",  # 替换为实际的API Key
    "Content-Type": "application/json"
}

# 上传训练数据
with open('/data/chats/mn0vvd/workspace/training_data.json', 'r', encoding='utf-8') as f:
    response = requests.post(url, headers=headers, data=f)

# 获取模型ID
model_id = response.json().get("model_id")
print(f"训练完成，模型ID为: {model_id}")

ConnectionError: HTTPSConnectionPool(host='tencent_api_gateway', port=443): Max retries exceeded with url: /deepseek/train (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fb1a02d7a30>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [4]:
import pandas as pd
import json
import requests

# 读取CSV文件
df = pd.read_csv('/data/chats/mn0vvd/workspace/chat_records.csv')

# 过滤出文本类型的消息
df = df[df['message_type'] == 'text']

# 初始化对话配对列表
paired_data = []

# 遍历数据，配对用户A和用户B的对话
prev_user, prev_message = None, None
for _, row in df.iterrows():
    if row['sender'] == '用户B' and prev_user == '用户A':
        paired_data.append({'input': prev_message, 'response': row['message']})
    prev_user, prev_message = row['sender'], row['message']

# 将配对后的对话保存为JSON文件，用于训练
with open('/data/chats/mn0vvd/workspace/training_data.json', 'w', encoding='utf-8') as f:
    json.dump(paired_data, f, ensure_ascii=False, indent=4)

# 调用腾讯云DeepSeek API进行训练
url = "https://api.tencentcloud.com/deepseek/train"  # 替换为正确的API地址
headers = {
    "X-API-Key": "your_api_key",  # 替换为实际的API Key
    "Content-Type": "application/json"
}

try:
    # 上传训练数据
    with open('/data/chats/mn0vvd/workspace/training_data.json', 'r', encoding='utf-8') as f:
        response = requests.post(url, headers=headers, data=f)
    
    # 检查响应状态码
    if response.status_code == 200:
        model_id = response.json().get("model_id")
        print(f"训练完成，模型ID为: {model_id}")
    else:
        print(f"API调用失败，状态码: {response.status_code}")
        print(f"错误信息: {response.text}")
except requests.exceptions.RequestException as e:
    print(f"请求失败: {e}")

UnicodeEncodeError: 'latin-1' codec can't encode characters in position 26-43: ordinal not in range(256)

In [5]:
import pandas as pd
import json
import requests

# 读取CSV文件
df = pd.read_csv('/data/chats/mn0vvd/workspace/chat_records.csv')

# 过滤出文本类型的消息
df = df[df['message_type'] == 'text']

# 初始化对话配对列表
paired_data = []

# 遍历数据，配对用户A和用户B的对话
prev_user, prev_message = None, None
for _, row in df.iterrows():
    if row['sender'] == '用户B' and prev_user == '用户A':
        paired_data.append({'input': prev_message, 'response': row['message']})
    prev_user, prev_message = row['sender'], row['message']

# 将配对后的对话保存为JSON文件，用于训练
with open('/data/chats/mn0vvd/workspace/training_data.json', 'w', encoding='utf-8') as f:
    json.dump(paired_data, f, ensure_ascii=False, indent=4)

# 调用腾讯云DeepSeek API进行训练
url = "https://api.tencentcloud.com/deepseek/train"  # 替换为正确的API地址
headers = {
    "X-API-Key": "your_api_key",  # 替换为实际的API Key
    "Content-Type": "application/json; charset=utf-8"  # 明确指定字符编码
}

try:
    # 读取JSON数据并确保编码为UTF-8
    with open('/data/chats/mn0vvd/workspace/training_data.json', 'r', encoding='utf-8') as f:
        json_data = json.dumps(json.load(f), ensure_ascii=False)
    
    # 上传训练数据
    response = requests.post(url, headers=headers, data=json_data.encode('utf-8'))
    
    # 检查响应状态码
    if response.status_code == 200:
        model_id = response.json().get("model_id")
        print(f"训练完成，模型ID为: {model_id}")
    else:
        print(f"API调用失败，状态码: {response.status_code}")
        print(f"错误信息: {response.text}")
except requests.exceptions.RequestException as e:
    print(f"请求失败: {e}")

API调用失败，状态码: 404
错误信息: 404 page not found
